In [6]:
import torch as th
import numpy as np

## General functions

In [ ]:
def rotation(theta):
    """Rotation matrix for a single qubit."""
    U = th.tensor(
            [[np.cos(theta / 2), -1j * np.sin(theta / 2)],
            [-1j * np.sin(theta / 2), np.cos(theta / 2)]],
            dtype=th.complex128)
    return U


def global_rotation(angles,N):
    """Apply a global rotation to the state vector."""

    U = rotation(angles[0])

    for i in range(1,N):
        U_i = rotation(angles[i])
        U = th.kron(U, U_i)
    return U


def cnot_gates(N):
    """Return a n-1 cascading CNOT."""

    cnot = th.tensor([
                [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 0, 1],
                [0, 0, 1, 0]
        ], dtype=th.complex128)
    
    cnot_gates = []
    
    for i in range(N):    
        p1 = i
        p2 = N-i-2

        Id1 = th.eye(2**p1, dtype=th.complex128)
        Id2 = th.eye(2**p2, dtype=th.complex128)

        cnot_gates.append(th.kron(Id1, th.kron(cnot, Id2)))
    
    return cnot_gates

def generate_ghz(N):
    state = th.zeros(2**N, dtype=th.complex128)
    state[0] = 1/np.sqrt(2)
    state[-1] = 1/np.sqrt(2)
    return state

## Density matrix computation

In [2]:
def generate_ghz_dm(N):
    """Generate a GHZ state for N qubits."""
    ghz_state = generate_ghz(N)
    return th.outer(ghz_state, ghz_state)


def generate_H(N):
    h = 9/N
    scale = 2**(N/2)

    Z = th.tensor([[1, 0], [0, -1]], dtype=th.complex128)

    ZZ = th.kron(Z, Z)

    H = th.zeros(2**N, 2**N, dtype=th.complex128)

    for i in range(N-1):
        H += th.kron( th.eye(2**i), th.kron(ZZ, th.eye(2**(N-i-2))) )

    return h*scale*H

In [3]:
def layer(state, angles, N):
    """Apply a layer of gates to the state vector."""
    U = global_rotation(angles,N)
    
    # Apply the rotation
    state = U @ state @ U.adjoint()
    

    CNOT_gates = cnot_gates(N)
    # Apply CNOT gates
    for cnot in CNOT_gates:
        state = cnot @ state @ cnot.adjoint()
    
    return state

def calc_variance(N,L,p):
    n_sim = 1

    ghz = generate_ghz_dm(N)
    H = generate_H(N)

    obs = 0

    for _ in range(n_sim):
        state = th.zeros(2**N,2**N, dtype=th.complex128)
        state[0,0] = 1.0  # |000>

        params = np.random.uniform(0, np.pi, size=(L, N))

        # Sample whether to entangle or use CNOT at each layer
        entangle_flags = np.random.rand(L) < p  # Boolean array

        for i, apply_entangled in enumerate(entangle_flags):
            if apply_entangled:
                state = ghz
            else:
                state = layer(state, params[i], N)

        # Run the circuit
        obs += th.trace(state @ H).real
    
    return obs/n_sim

In [254]:
calc_variance(10, 1, 0.1)

tensor(-88.7017, dtype=torch.float64)

## Optimized state vector computation

In [ ]:
def layer(state, angles, N):
    """Apply a layer of gates to the state vector."""
    U = global_rotation(angles,N)
    
    # Apply the rotation
    state = U @ state

    CNOT_gates = cnot_gates(state, N, 0) 
    
    # Apply CNOT gates
    for cnot in CNOT_gates:
        state = cnot @ state
    
    return state


def generate_dm(N,L,p):
    # Sample whether to entangle or use CNOT at each layer
    entangle_flags = np.random.rand(L) < p  # Boolean array

    last_true_idx = np.where(entangle_flags)[0][-1] if np.any(entangle_flags) else 0

    l = L - last_true_idx   # number of effective layers
    
    params = np.random.uniform(0, np.pi, size=(l, N))
    
    state = generate_ghz_dm(N) 

    for i in range(l):
        state = layer(state, params[i], N)


def calc_variance(N,L,p):
    n_sim = 1

    H = generate_H(N)

    res = []

    for _ in range(n_sim):
        
        state = generate_dm(N,L,p)

        # Run the circuit
        res.append(th.trace(state @ H).real)
    
    return np.std(res)